# HVAC vs ERCOT 4CP event alignment:
This notebook explores how household cooling aligns with the State of Texas’s critical peak power events.

We will be using one-minute interval data from 25 Texas homes and will determine what percentage of homes 
had air conditioning on during these CP events for each year from 2016-2019.

To prepare the data we selected 25 Texas homes with PV and hvac system. We used 1 minute energy(from electricity.eg_realpower_1min) data for the selected homes for the CP dates mentioned below.

In [ ]:
# These are the ERCOT 4CP events (start date/time and end date/time) for 2016 - 2019 acquired from
# http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13037&reportTitle=Planned%20Service%20Four%20Coincident%20Peak%20Calculations&showHTMLView=&mimicKey

event_start_dates = ['2019-06-19 17:00:00-05', '2019-07-30 16:30:00-05', '2019-08-12 17:00:00-05', '2019-09-06 16:45:00-05',
               '2018-06-27 17:00:00-05', '2018-07-19 17:00:00-05', '2018-08-23 16:45:00-05', '2018-09-19 16:30:00-05',
               '2017-06-23 16:45:00-05', '2017-07-28 17:00:00-05', '2017-08-16 17:00:00-05', '2017-09-20 16:45:00-05',
               '2016-06-15 17:00:00-05', '2016-07-14 16:00:00-05', '2016-08-11 16:30:00-05', '2016-09-19 16:15:00-05'
              ]
event_end_dates = ['2019-06-19 17:15:00-05', '2019-07-30 16:45:00-05', '2019-08-12 17:15:00-05', '2019-09-06 17:00:00-05',
               '2018-06-27 17:15:00-05', '2018-07-19 17:15:00-05', '2018-08-23 17:00:00-05', '2018-09-19 16:45:00-05',
               '2017-06-23 17:00:00-05', '2017-07-28 17:15:00-05', '2017-08-16 17:15:00-05', '2017-09-20 17:00:00-05',
               '2016-06-15 17:15:00-05', '2016-07-14 16:15:00-05', '2016-08-11 16:45:00-05', '2016-09-19 16:30:00-05']

In [ ]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
import numpy as np
import sys
%matplotlib inline
sys.executable  # shows you your path to the python you're using

In [ ]:
#Read pre-processed data 
df3 = pd.read_csv('/shared/JupyterHub-Examples-Data/hvac/hvac_cooling_4cp.zip', compression='zip')

homes_list = df3.dataid.unique() 

In [ ]:
#replace null with 0's 
df3 = df3.fillna(0)
df3.head(10)

In [ ]:
#create new column hvac.(hvac is sum of air compressor and furnace circuits.)
df3['hvac'] = df3['air1'] + df3['air2'] + df3['furnace1'] + df3['furnace2']
df3['localminute'] = pd.to_datetime(df3['localminute'])
df3['localminute'] = df3['localminute'].dt.date
df3.head(10)

In [ ]:
#create new dataframe with columns dataid,hvac and date.
hvac_cp_data = pd.DataFrame(df3, columns = ['dataid','hvac', 'localminute'])
hvac_cp_data.head(10)

In [ ]:
#group data by dataid and date and take average of hvac. 
#This gives us avaerage hvac usage for each home for each CP event day.
hvac_cp_data_grouped = hvac_cp_data.groupby(['localminute','dataid']).mean()
reset_hvac_cp_data_grouped = hvac_cp_data_grouped.reset_index()
reset_hvac_cp_data_grouped.head(10)

In [ ]:
#filtering data only for those interval when AC is actually cooling..
#We consider AC is ON and cooling only when it's drawing power of atleast 500W. 

hvac_on = reset_hvac_cp_data_grouped[reset_hvac_cp_data_grouped['hvac'] > 0.5]
homes_on = (hvac_on['localminute'].value_counts()/25)*100
homes_on_perc = pd.DataFrame(homes_on.reset_index().values, columns=["Date", "Percentage"])
homes_on_percindex = homes_on_perc.sort_index(axis = 0, ascending=True)
homes_on_percindex

In [ ]:
#plot bar chart
homes_on_percindex.plot.bar(x='Date', y='Percentage',figsize=(20,10),color='orange',title="Home's cooling alignment with Texas CP events")
plt.show()

In [ ]:
homes_on_percindex['Percentage'].mean()

## From above results we can see that nearly 86% (85.5%) of homes were ON and cooling for each 4CP event day from 2016-2019.